# Explore here

In [ ]:
 # your code here

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
url = 'https://raw.githubusercontent.com/4GeeksAcademy/regularized-linear-regression-project-tutorial/main/demographic_health_data.csv'
df = pd.read_csv(url)

# Show columns to confirm column names
print("Columns in dataset:")
print(df.columns.tolist())

# Check for missing values per column
print("\nMissing values per column:")
print(df.isnull().sum())

# Drop irrelevant or non-numeric columns
# 'Unnamed: 0' is an index column; 'County Name' and 'State' are categorical labels not useful for numeric modeling here
cols_to_drop = ['Unnamed: 0', 'County Name', 'State']
existing_cols_to_drop = [col for col in cols_to_drop if col in df.columns]
df = df.drop(columns=existing_cols_to_drop)

# Drop columns with more than 30% missing values
# Features with excessive missing data can reduce model reliability and require heavy imputation, so we remove them
threshold = len(df) * 0.3
cols_before = df.shape[1]
df = df.loc[:, df.isnull().sum() <= threshold]
cols_after = df.shape[1]
print(f"\nDropped {cols_before - cols_after} columns with >30% missing values")

# For remaining missing values, fill with column mean to keep numeric data consistent
numeric_cols = df.select_dtypes(include=np.number).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

# Define target variable
target = 'Smoking'  # Confirmed column name for smokers %

# Separate features and target
X = df.drop(columns=[target])
y = df[target]

# Split into training and testing sets for model evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit baseline linear regression model
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
r2_lr = r2_score(y_test, y_pred_lr)
print(f"\nLinear Regression R2 score: {r2_lr:.4f}")

# Fit Lasso regression and analyze effect of alpha on R2
alphas = np.linspace(0, 20, 40)
r2_scores = []

for alpha in alphas:
    lasso = Lasso(alpha=alpha, max_iter=10000)
    lasso.fit(X_train, y_train)
    y_pred_lasso = lasso.predict(X_test)
    r2 = r2_score(y_test, y_pred_lasso)
    r2_scores.append(r2)

# Plot R2 scores vs alpha values
plt.figure(figsize=(10, 6))
plt.plot(alphas, r2_scores, marker='o')
plt.title('Lasso Regression: R2 Score vs Alpha')
plt.xlabel('Alpha')
plt.ylabel('R2 Score')
plt.grid(True)
plt.show()

# Pick best alpha based on highest R2
best_alpha = alphas[np.argmax(r2_scores)]
print(f"\nBest alpha for Lasso: {best_alpha:.2f} with R2 = {max(r2_scores):.4f}")

# Train final Lasso model with best alpha
lasso_final = Lasso(alpha=best_alpha, max_iter=10000)
lasso_final.fit(X_train, y_train)
y_pred_final = lasso_final.predict(X_test)
print(f"\nFinal Lasso model R2 score: {r2_score(y_test, y_pred_final):.4f}")


Columns in dataset:
['fips', 'TOT_POP', '0-9', '0-9 y/o % of total pop', '19-Oct', '10-19 y/o % of total pop', '20-29', '20-29 y/o % of total pop', '30-39', '30-39 y/o % of total pop', '40-49', '40-49 y/o % of total pop', '50-59', '50-59 y/o % of total pop', '60-69', '60-69 y/o % of total pop', '70-79', '70-79 y/o % of total pop', '80+', '80+ y/o % of total pop', 'White-alone pop', '% White-alone', 'Black-alone pop', '% Black-alone', 'Native American/American Indian-alone pop', '% NA/AI-alone', 'Asian-alone pop', '% Asian-alone', 'Hawaiian/Pacific Islander-alone pop', '% Hawaiian/PI-alone', 'Two or more races pop', '% Two or more races', 'POP_ESTIMATE_2018', 'N_POP_CHG_2018', 'GQ_ESTIMATES_2018', 'R_birth_2018', 'R_death_2018', 'R_NATURAL_INC_2018', 'R_INTERNATIONAL_MIG_2018', 'R_DOMESTIC_MIG_2018', 'R_NET_MIG_2018', 'Less than a high school diploma 2014-18', 'High school diploma only 2014-18', "Some college or associate's degree 2014-18", "Bachelor's degree or higher 2014-18", 'Percen

TypeError: Could not convert ["AutaugaBaldwinBarbourBibbBlountBullockButlerCalhounChambersCherokeeChiltonChoctawClarkeClayCleburneCoffeeColbertConecuhCoosaCovingtonCrenshawCullmanDaleDallasDeKalbElmoreEscambiaEtowahFayetteFranklinGenevaGreeneHaleHenryHoustonJacksonJeffersonLamarLauderdaleLawrenceLeeLimestoneLowndesMaconMadisonMarengoMarionMarshallMobileMonroeMontgomeryMorganPerryPickensPikeRandolphRussellSt. ClairShelbySumterTalladegaTallapoosaTuscaloosaWalkerWashingtonWilcoxWinstonAleutians WestAnchorageBethelBristol BayDenaliDillinghamFairbanks North StarHainesHoonah-AngoonJuneauKenai PeninsulaKetchikan GatewayKodiak IslandKusilvakLake and PeninsulaMatanuska-SusitnaNomeNorth SlopeNorthwest ArcticPetersburgPrince of Wales-HyderSitkaSkagwaySoutheast FairbanksWrangellYakutatYukon-KoyukukApacheCochiseCoconinoGilaGrahamGreenleeLa PazMaricopaMohaveNavajoPimaPinalSanta CruzYavapaiYumaArkansasAshleyBaxterBentonBooneBradleyCalhounCarrollChicotClarkClayCleburneClevelandColumbiaConwayCraigheadC